In [1]:
import os
import pandas as pd
import numpy as np
import pandas as pd
from sklearn import preprocessing

import matplotlib.pyplot as plt
plt.rc("font", size=14)
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [2]:
import cv2
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.autograd import Variable
#from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv2d, MaxPool2d, Module, Softmax, BatchNorm2d, Dropout
from torch.optim import Adam, SGD

In [3]:
from torch.utils.data import DataLoader

In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [5]:
train_img = []
train_labels = []

test_img = []
test_labels = []

#path_train = ('/Users/johndufek/Downloads/archive (2)/Training/')# code for local directory (e.g. in jupyter)
#path_test = ('/Users/johndufek/Downloads/archive (2)/Testing/')
path_train = ('/content/gdrive/MyDrive/archive (2)/Training/')
path_test = ('/content/gdrive/MyDrive/archive (2)/Testing/')

img_size= 300

for i in os.listdir(path_train):
    for j in os.listdir(path_train+i):
        train_img.append (cv2.resize(cv2.imread(path_train+i+'/'+j), (img_size,img_size)))
        train_labels.append(i)

for i in os.listdir(path_test):
    for j in os.listdir(path_test+i):
        test_img.append (cv2.resize(cv2.imread(path_test+i+'/'+j), (img_size,img_size)))
        test_labels.append(i)

train_img = (np.array(train_img))
test_img = (np.array(test_img))


train_labels_encoded = [0 if category == 'no_tumor' else(1 if category == 'glioma_tumor' else(2 if category=='meningioma_tumor' else 3)) for category in list(train_labels)]
test_labels_encoded = [0 if category == 'no_tumor' else(1 if category == 'glioma_tumor' else(2 if category=='meningioma_tumor' else 3)) for category in list(test_labels)]



In [1]:
#os.listdir('/content/gdrive/MyDrive/archive (2)/Training/')

In [6]:
# Configure Device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyper params
epochs = 30
#batch_size
lrn_rate = 0.005


In [7]:
# Transform function to normalize photo data
transform = transforms.Compose([
    transforms.Resize((150,150)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.5,0.5,0.5],
                        [0.5,0.5,0.5])
])

In [8]:
# Create Training and Validation data sets
train_x, val_x, train_y, val_y = train_test_split(np.array(train_img), np.array(train_labels), test_size = 0.20)

In [9]:
# Normalize image data
train_load=DataLoader(
    torchvision.datasets.ImageFolder(path_train,transform=transform),
    batch_size=64, shuffle=True
)
test_load=DataLoader(
    torchvision.datasets.ImageFolder(path_test,transform=transform),
    batch_size=32, shuffle=True
)

In [10]:
# Actual Network With Nodes
class CNN(nn.Module):
  def __init__(self):
    super(CNN,self).__init__()
    self.conv1 = nn.Conv2d(3, 15, 3)
    self.pool = nn.MaxPool2d(2, 2)
    self.conv2 = nn.Conv2d(15, 25, 3)
    self.conv3 = nn.Conv2d(25, 30, 3)
    self.fc1 = nn.Linear(30*17*17, 300)
    self.fc2 = nn.Linear(300, 100)
    self.fc3 = nn.Linear(100, 4)
    self.bn= nn.BatchNorm2d(num_features=15)
    self.bn2=nn.BatchNorm2d(num_features=30)


  def forward(self, x):
    x = self.pool(F.relu(self.bn(self.conv1(x))))
    x = self.pool(F.relu(self.conv2(x)))
    x = self.pool(F.relu(self.bn2(self.conv3(x))))
    x = x.view(-1,30*17*17)
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)

    return x


In [11]:
dataiter = iter(train_load)
images,labels = next(dataiter)


In [10]:
conv1 = nn.Conv2d(3, 15, 3)
pool = nn.MaxPool2d(2, 2)
conv2 = nn.Conv2d(15, 25, 3)
conv3 = nn.Conv2d(25, 30, 3)

In [11]:
# Figure out number of inputs going into the first linear layer.
x = conv1(images)
x = pool(x)
x = conv2(x)
x = pool(x)
x = conv3(x)
x = pool(x)
print(x.shape)

torch.Size([64, 30, 17, 17])


In [12]:
# Cross Entropy Loss and Stochastic Gradient Descent
model = CNN().to(device)
loss_f = nn.CrossEntropyLoss()
opt = torch.optim.Adam(model.parameters(), lr = lrn_rate)

n = len(train_load)
for epoch in range(epochs):
  train_acc = 0
  train_loss = 0
  model.train()

  for (images,labels) in iter(train_load):
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())

        opt.zero_grad()

        x=model(images)
        loss=loss_f(x,labels)
        loss.backward()
        opt.step()


        train_loss+= loss.cpu().data*images.size(0)
        _,prediction=torch.max(x.data,1)

        train_acc+=int(torch.sum(prediction==labels.data))

  train_acc=train_acc/n
  train_loss=train_loss/n

  model.eval()

  test_acc=0.0
  for (images,labels) in iter(test_load):
      if torch.cuda.is_available():
          images=Variable(images.cuda())
          labels=Variable(labels.cuda())

      x=model(images)
      _,prediction=torch.max(x.data,1)
      test_acc+=int(torch.sum(prediction==labels.data))

  test_acc=test_acc/len(test_load)


  print('Epoch: '+str(epoch)+' Train Loss: '+str(train_loss)+' Train Accuracy: '+str(train_acc)+' Test Accuracy: '+str(test_acc))


Epoch: 0 Train Loss: tensor(125.0634) Train Accuracy: 36.28888888888889 Test Accuracy: 8.615384615384615
Epoch: 1 Train Loss: tensor(45.2104) Train Accuracy: 45.08888888888889 Test Accuracy: 12.23076923076923
Epoch: 2 Train Loss: tensor(38.9879) Train Accuracy: 47.82222222222222 Test Accuracy: 12.615384615384615
Epoch: 3 Train Loss: tensor(32.9402) Train Accuracy: 50.62222222222222 Test Accuracy: 15.692307692307692
Epoch: 4 Train Loss: tensor(30.5243) Train Accuracy: 50.91111111111111 Test Accuracy: 15.153846153846153
Epoch: 5 Train Loss: tensor(27.0825) Train Accuracy: 52.266666666666666 Test Accuracy: 16.0
Epoch: 6 Train Loss: tensor(22.5856) Train Accuracy: 54.68888888888889 Test Accuracy: 15.23076923076923
Epoch: 7 Train Loss: tensor(22.4229) Train Accuracy: 55.13333333333333 Test Accuracy: 17.76923076923077
Epoch: 8 Train Loss: tensor(18.6100) Train Accuracy: 56.68888888888889 Test Accuracy: 19.153846153846153
Epoch: 9 Train Loss: tensor(16.4322) Train Accuracy: 57.68888888888889 